<a href="https://colab.research.google.com/github/wockeshuh/CSCI164/blob/main/Copy_of_AI24Ch3a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Freddy Barrita
# 7/31/24


In [108]:
import random
import heapq


# Tile Sliding Domain: Initial State Space

In [109]:
StateDimension=3
InitialState = [1,2,3,4,5,6,7,8,13,14,9,12,10,11,15,0]
GoalState=[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0]
Actions = lambda s: ['u', 'd', 'l', 'r']
Opposite=dict([('u','d'),('d','u'),('l','r'),('r','l'), (None, None)])

In [110]:
def Result(state, action):
  newState = list(state).copy()
  row, col = divmod(newState.index(0), StateDimension)
  if action=='u' :
    l,r = row*StateDimension+col, (row-1)*StateDimension+col
  elif action=='d' :
    l,r = row*StateDimension+col, (row+1)*StateDimension+col
  elif action=='l' :
    l,r = row*StateDimension+col, row*StateDimension+col-1
  elif action=='r' :
    l,r = row*StateDimension+col, row*StateDimension+col+1
  # Check if indices are within bounds before swapping
  if 0 <= l < len(newState) and 0 <= r < len(newState):
    newState[l], newState[r] = newState[r], newState[l]
  return tuple(newState)

def PrintState(s):
  for i in range(0,len(s),StateDimension):
    print(s[i:i+StateDimension])

def LegalMove(state, action):
  i = state.index(0)
  row,col=i//StateDimension, i % StateDimension
  newState = state.copy()
  if ( (action=='u' and row==0) or
       (action=='d' and row==StateDimension-1) or
       (action=='l' and col==0) or
       (action=='r' and col==StateDimension-1)):
      return False
  return True


In [111]:
PrintState(InitialState)

[1, 2, 3]
[4, 5, 6]
[7, 8, 13]
[14, 9, 12]
[10, 11, 15]
[0]


In [112]:
PrintState(GoalState)

[1, 2, 3]
[4, 5, 6]
[7, 8, 9]
[10, 11, 12]
[13, 14, 15]
[0]


In [113]:
PrintState(InitialState)
print()
state1 = Result(InitialState, 'u')
PrintState(state1)
print()
state1 = Result(state1, 'r')
PrintState(state1)

[1, 2, 3]
[4, 5, 6]
[7, 8, 13]
[14, 9, 12]
[10, 11, 15]
[0]

(1, 2, 3)
(4, 5, 6)
(7, 8, 13)
(14, 9, 12)
(0, 11, 15)
(10,)

(1, 2, 3)
(4, 5, 6)
(7, 8, 13)
(14, 9, 12)
(11, 0, 15)
(10,)


In [114]:
def ApplyMoves(actions, state):
  for action in actions:
    state = Result(state, action)
  return state

In [115]:
PrintState(InitialState)
print(['r','r'])
PrintState(ApplyMoves(['r','r'],InitialState))

[1, 2, 3]
[4, 5, 6]
[7, 8, 13]
[14, 9, 12]
[10, 11, 15]
[0]
['r', 'r']
(1, 2, 3)
(4, 5, 6)
(7, 8, 13)
(14, 9, 12)
(10, 11, 15)
(0,)


## Problem Class

INITIAL = InitialState  
IsGoal = Goal Test  
Actions = Actions List  
Result = Action Behavior  
ActionCost = Action Cost  

In [116]:
class Problem(object): pass

## Node

In [117]:
class Node(object):
  def __init__(self, state, parent=None, action=None, path_cost=0 ):
    self.State=state
    self.Parent=parent
    self.Action=action
    self.PathCost = path_cost

  def __str__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __repr__(self):
    action = "<none>" if not self.Action else self.Action
    return str(self.State) + ", " + action
  def __lt__(self, other):
    return self.PathCost < other.PathCost;

## Expand

In [118]:
def Expand(problem, node):
  ret = []
  s = node.State
  for action in problem.Actions(s):
    sPrime = problem.Result(s, action)
    cost =node.PathCost + problem.ActionCost(s,action,sPrime)
    ret.append(Node(sPrime, node, action, cost))
  return ret


## Breadth-First Search

In [119]:
def BreadthFirstSearch(problem):
  node = Node(tuple(problem.INITIAL))
  if problem.IsGoal(node.State):
    return node, 0
  Frontier = []
  Frontier.append(node)
  reached = set()
  reached.add(tuple(problem.INITIAL))
  nodesExpanded = 0
  while (Frontier):
    ### print([str(n) for n in Frontier])
    node = Frontier.pop(0)
    ### print(node)
    for child in Expand(problem, node):
      s = tuple(child.State)
      ### print (s, "IsGoal=", problem.IsGoal(s))
      if problem.IsGoal(s):
        return child, nodesExpanded
      if s not in reached:
        reached.add(s)
        Frontier.append(child)
    nodesExpanded += 1
    if nodesExpanded > 500000:
      break;
  return None, nodesExpanded

## Best-First Search

In [120]:
def BestFirstSearch(problem, f):
  node = Node(tuple(problem.INITIAL))
  Frontier = []
  heapq.heappush(Frontier,(f(node), node))
  reached = {}
  reached[tuple(problem.INITIAL)]=node
  nodesExpanded = 0
  while (Frontier):
    ##print([(x, str(n)) for (x,n) in Frontier])
    fValue, node = heapq.heappop(Frontier)
    ##print (node.State, "IsGoal=", problem.IsGoal(tuple(node.State)))
    if problem.IsGoal(tuple(node.State)):
      return node, nodesExpanded    ### print(node)
    for child in Expand(problem, node):
      s = tuple(child.State)
      if s not in reached or child.PathCost < reached[s].PathCost:
        reached[s] = child
        heapq.heappush(Frontier, (f(child), child))
    nodesExpanded += 1
    if nodesExpanded > 500000:
      break;
  return None, nodesExpanded

## Problem 1

In [121]:
TileSliding = Problem()
TileSliding.INITIAL = InitialState
TileSliding.IsGoal = lambda s: s==(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)
TileSliding.Actions = Actions
TileSliding.Result=Result
TileSliding.ActionCost = lambda s, a, sPrime: 1
print( TileSliding.IsGoal((1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,0)) )
print( Node(InitialState) )
print(1+TileSliding.ActionCost(1,2,3))

True
[1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 9, 12, 10, 11, 15, 0], <none>
2


In [122]:
TileSliding.INITIAL = [1,2,3,4,5,6,7,8,13,14,9,12,10,11,15,0]
ret, cost = BreadthFirstSearch(TileSliding)
print (ret, cost)

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0), r 136743


In [123]:
def Solution(node):
  if node.Parent==None:
    return []
  return Solution(node.Parent) + [node.Action]


In [124]:
sol = Solution(ret)
print (sol)
print (TileSliding.INITIAL)
print (ApplyMoves(sol, TileSliding.INITIAL))


['u', 'u', 'r', 'd', 'l', 'd', 'l', 'u', 'u', 'r', 'd', 'l', 'd', 'r']
[1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 9, 12, 10, 11, 15, 0]
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)


In [125]:
TileSliding.INITIAL = [1,2,3,4,5,6,7,8,13,14,9,12,10,11,15,0]
ret, cost = BreadthFirstSearch(TileSliding)
print (ret, cost)

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0), r 136743


In [126]:
sol = Solution(ret)
cost = len(sol)
print("Action Sequence: ", sol)
print("Before moves: ",TileSliding.INITIAL)
print("After moves: ", ApplyMoves(sol, TileSliding.INITIAL))
print("Cost of the action sequence:", cost)

Action Sequence:  ['u', 'u', 'r', 'd', 'l', 'd', 'l', 'u', 'u', 'r', 'd', 'l', 'd', 'r']
Before moves:  [1, 2, 3, 4, 5, 6, 7, 8, 13, 14, 9, 12, 10, 11, 15, 0]
After moves:  (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)
Cost of the action sequence: 14
